<a href="https://colab.research.google.com/github/mlukan/GDA3B2021/blob/main/Martin_Lukan_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.	Import this dataset of tweets into a DataFrame.
2.	Keep only positive and negative tweets (so you exclude neutrals). What is the percentage of positive/negative tweets?

url='https://github.com/murpi/wilddata/raw/master/quests/tweets.zip'

In [2]:
url="https://github.com/murpi/wilddata/raw/master/quests/tweets.zip"
import requests, zipfile, io
import os
r = requests.get(url)

z = zipfile.ZipFile(io.BytesIO(r.content))
filepath=os.getcwd()+"/files/"
z.extractall(filepath)

In [3]:
import pandas as pd
filename= z.filelist[0].filename
tweetsdf=pd.read_csv(filepath+filename)
print(len(tweetsdf))
tweetsdf=tweetsdf[tweetsdf['sentiment']!='neutral']
tweetsdf.describe()

27481


,textID,text,selected_text,sentiment
count,16363,16363,16363,16363
unique,16363,16363,11377,2
top,db7dc72889,g`mornin,good,positive
freq,1,1,199,8582


In [10]:
tweetsdf['sentiment'].value_counts(normalize=True)*100

positive    52.447595
negative    47.552405
Name: sentiment, dtype: float64

3.	Retrieve the list of English stopwords from NLTK, and copy it into a stopwordsenglish list.
4.	Create a lemma function that takes a text (str) as a parameter and returns a text (str) of tokens after applying a stemmer or a lemmatizer, separated by spaces.
5.	Apply this function lemma to the text column of your DataFrame. Store the result in a new lemma column of the DataFrame. (The processing can take 2 or 3 minutes).


In [ ]:
import nltk
import string
nltk.download('stopwords') 
from nltk.corpus import stopwords
stopwordsenglish=stopwords.words('english')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemma(sentence):
    return " ".join([lemmatizer.lemmatize(word) for word in sentence.split()])

tweetsdf['lemma']=tweetsdf['text'].apply(lambda x: lemma(x))
    




[nltk_data] Downloading package stopwords to
[nltk_data]     /home/A19893678/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/A19893678/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
tweetsdf

,textID,text,selected_text,sentiment,lemma
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,Sooo SAD I will miss you here in San Diego!!!
2,088c60f138,my boss is bullying me...,bullying me,negative,my bos is bullying me...
3,9642c003ef,what interview! leave me alone,leave me alone,negative,what interview! leave me alone
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"Sons of ****, why couldn`t they put them on th..."
6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,positive,2am feeding for the baby are fun when he is al...
...,...,...,...,...,...
27475,b78ec00df5,enjoy ur night,enjoy,positive,enjoy ur night
27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative,wish we could come see u on Denver husband los...
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative,I`ve wondered about rake to. The client ha mad...
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive,Yay good for both of you. Enjoy the break - yo...


 
6.	Copy the lemma column into a Series X, and the sentiment column into a Series y. Apply a train test split with the random_state = 32.  


In [ ]:
X,y=tweetsdf['lemma'],tweetsdf['sentiment']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=32)

7.	Create a vectorizer model with scikit-learn using the Countvectorizer method. Train your model on X_train, then create a matrix of features X_train_CV. Create the X_test_CV matrix without re-training the model. The format of the X_test_CV matrix should be 4091x13838 with 44274 stored elements.  


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression(max_iter=1000)
count_vectorizer = CountVectorizer()
cvec = count_vectorizer.fit(X_train)
X_train_cvec = cvec.transform(X_train)
lr.fit(X_train_cvec,y_train)

LogisticRegression(max_iter=1000)

In [ ]:
cvec_score=cross_val_score(lr,X_train_cvec,y_train,cv=3)
cvec_score

array([0.87362503, 0.8665363 , 0.87066015])

In [ ]:
X_train_cvec

<12272x15392 sparse matrix of type '<class 'numpy.int64'>'
	with 144148 stored elements in Compressed Sparse Row format>

In [ ]:
X_test_CV=cvec.transform(X_test)
X_test_CV

<4091x15392 sparse matrix of type '<class 'numpy.int64'>'
	with 44554 stored elements in Compressed Sparse Row format>

8.	Using X_train_CV and y_train, please run a logistic regression to classify tweets. You should get a score of 0.87 on the X_test_CV test set.  


In [ ]:
cross_val_score(lr,X_test_CV,y_test,cv=3)

array([0.83870968, 0.83577713, 0.83639032])

9.	Repeat steps 7 and 8, adding the stop_words parameter to the CountVectorizer, using your stopwordsenglish list from step 3. Is the score very different? Is the number of elements stored in the sparse matrix very different?  

In [ ]:

lr1=LogisticRegression()
count_vectorizer1 = CountVectorizer(stop_words='english')
X_train_cvec1 = count_vectorizer1.fit_transform(X_train)
#X_train_cvec1 = cvec1.transform(X_train)
lr1.fit(X_train_cvec1,y_train)

LogisticRegression()

In [ ]:
X_train_cvec1

<12272x15118 sparse matrix of type '<class 'numpy.int64'>'
	with 83163 stored elements in Compressed Sparse Row format>

In [ ]:
X_test_CV1=cvec1.transform(X_test)
print('Cross val score:',cross_val_score(lr1,X_train_cvec1,y_train,cv=3).mean())
print('Accuracy score:', accuracy_score(y_train,lr1.predict(X_train_cvec1)))


Cross val score: 0.860332488853322
Accuracy score: 0.9586049543676662


In [ ]:
X_test_CV1

<4091x15118 sparse matrix of type '<class 'numpy.int64'>'
	with 24569 stored elements in Compressed Sparse Row format>

10.	Test the model using TfidfVectorizer instead of Countvectorizer, and repeat steps 7 and 8. Which preprocessing method gives you the best results on the test set? Are the scores very different on this dataset?  

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
lrtfidf=LogisticRegression()
tfidfvec = tfidf.fit(X_train)
X_train_tfidf = tfidfvec.transform(X_train)
lrtfidf.fit(X_train_tfidf,y_train)

LogisticRegression()

In [ ]:
print('Cross val score:', cross_val_score(lrtfidf,X_train_tfidf,y_train,cv=3).mean())
print('Accuracy score:', accuracy_score(y_train,lrtfidf.predict(X_train_tfidf)))


Cross val score: 0.8665255215645212
Accuracy score: 0.9310625814863103


In [ ]:
X_test_tfidf=tfidfvec.transform(X_test)
print(cross_val_score(lrtfidf,X_test_tfidf,y_test,cv=3))

[0.8313783  0.83064516 0.84079237]


## Using pipeline

In [ ]:
from sklearn.pipeline import Pipeline 
from sklearn.metrics import accuracy_score, log_loss

pipe = Pipeline(steps=[("cbow", CountVectorizer()), 
                       ("lr", LogisticRegression(random_state=32,max_iter=1000))])

pipe.fit(X_train, y_train)

print("Accuracy")
print(accuracy_score(y_train, pipe.predict(X_train)))
print(cross_val_score(pipe, X_train, y_train, cv=5).mean())
print(accuracy_score(y_test, pipe.predict(X_test)))


print("\nLogloss")
print(log_loss(y_train, pipe.predict_proba(X_train)[:,1]))
print(-1*cross_val_score(pipe,X_train, y_train, cv=5, scoring="neg_log_loss").mean())
print(log_loss(y_test, pipe.predict_proba(X_test)[:,1]))

Accuracy
0.9658572359843546
0.8702734635002998
0.8731361525299438

Logloss
0.16128167549530212
0.31527710807440656
0.3098536173482783


In [ ]:
from sklearn.pipeline import Pipeline 
from sklearn.metrics import accuracy_score, log_loss

pipe = Pipeline(steps=[("tfidf", TfidfVectorizer()), 
                       ("lr", LogisticRegression(random_state=32))])

pipe.fit(X_train, y_train)

print("Accuracy")
print(accuracy_score(y_train, pipe.predict(X_train)))
print(cross_val_score(pipe, X_train, y_train, cv=5).mean())
print(accuracy_score(y_test, pipe.predict(X_test)))


print("\nLogloss")
print(log_loss(y_train, pipe.predict_proba(X_train)[:,1]))
print(-1*cross_val_score(pipe,X_train, y_train, cv=5, scoring="neg_log_loss").mean())
print(log_loss(y_test, pipe.predict_proba(X_test)[:,1]))

Accuracy
0.9310625814863103
0.8697847647217974
0.8692251283304816

Logloss
0.30002890875497806
0.36118196753688375
0.3444881340684517


11.	Bonus step: try to display 10 tweets that were badly predicted (false positive or false negative). Would you have done better than the algorithm  

In [ ]:
len(y_test)

4091

In [ ]:
predf=pd.concat([X_test,y_test],axis=1)

In [ ]:
predf['predictions']=predictions.tolist()
#X_test
#y_test

#predictions

In [ ]:
pd.set_option("display.max_colwidth", None)
predf[predf['predictions']!=predf['sentiment']][:19]

,lemma,sentiment,predictions
18732,"SUFFICATION NO BREATHING. It`s okay. There`ll be more. You`re invited to mine, but I can`t promise fun times. *Jinx",negative,positive
12055,i wanna vote for Miley Cyrus for the mtv movie awards..but i don`t know where i could somebody could send me a link? thaank you <3,positive,negative
21824,I love music so much that i`ve gone through pain to play :S my side of my finger now are peeling and have blister from playing so much,negative,positive
18465,"I can only message those who message me, if we`re fwends...so those that want replies..follow me. hmm..that sound funny..",positive,negative
2976,"wish I could feel no pain (8) but it`s ok, at least they like Brazil!",positive,negative
3922,so glad i`m not at uni anymore,positive,negative
5199,You`re not here. I hope you`re still resting. I don`t want you to be stressed.,positive,negative
468,"you`re missing out, bb! i`m such a cereal nut, i think i like every kind available.",positive,negative
15216,have an amazing time with your momma tomorrow! Show them how much they mean to you Whatever you do they will love it,negative,positive
2977,hoping i didn`t fail english. that would just be sad,positive,negative
